In [1]:
import pandas as pd
import numpy as np
categories_df = pd.read_csv('../data/amazon_categories.csv')
products_df = pd.read_csv('../data/amazon_products.csv')
data = pd.merge(categories_df, products_df, left_on='id', right_on='category_id')
data = data.drop('id', axis=1)

In [2]:
data.head(10)

,category_name,asin,title,imgUrl,productURL,stars,reviews,price,listPrice,category_id,isBestSeller,boughtInLastMonth
0,Beading & Jewelry Making,B08YNTXFJG,Duufin 120 Pieces Acrylic Keychain Blanks Set ...,https://m.media-amazon.com/images/I/71czFH2QUM...,https://www.amazon.com/dp/B08YNTXFJG,4.7,510,9.99,0.00,1,False,50
1,Beading & Jewelry Making,B08DXSF5PZ,"DIY Tray Silicone Resin Mold, Oval Epoxy Resin...",https://m.media-amazon.com/images/I/71ZnrCHA49...,https://www.amazon.com/dp/B08DXSF5PZ,4.6,584,8.99,0.00,1,False,0
2,Beading & Jewelry Making,B096JGXQN5,"GTOTd I Love Rock and Roll Pins Set(18 Pack,2....",https://m.media-amazon.com/images/I/71SOJXp-wJ...,https://www.amazon.com/dp/B096JGXQN5,4.5,245,12.79,0.00,1,False,50
3,Beading & Jewelry Making,B0BZXW1ZGS,Xuhal 100 Pack Plastic Ring Sizer Bulk Reusabl...,https://m.media-amazon.com/images/I/81qD7tFV+r...,https://www.amazon.com/dp/B0BZXW1ZGS,5.0,1,16.99,0.00,1,False,0
4,Beading & Jewelry Making,B0BR8B472F,"Large Dice Resin Molds, 2 Styles Silicone Dice...",https://m.media-amazon.com/images/I/71uDVuhf71...,https://www.amazon.com/dp/B0BR8B472F,4.4,10,15.99,0.00,1,False,50
5,Beading & Jewelry Making,B096HTDZR5,10 Pieces 25 mm Bezel Tray Blank Bangle Bracel...,https://m.media-amazon.com/images/I/71M5IJaWA2...,https://www.amazon.com/dp/B096HTDZR5,4.4,130,14.99,0.00,1,False,0
6,Beading & Jewelry Making,B07SVCV8VB,100pcs Adabele Authentic 925 Sterling Silver C...,https://m.media-amazon.com/images/I/51wAAOtlbz...,https://www.amazon.com/dp/B07SVCV8VB,4.5,82,10.49,0.00,1,False,0
7,Beading & Jewelry Making,B0B4689HLD,90 Pcs Antique Silver Jesus Mary Religious Ben...,https://m.media-amazon.com/images/I/713kgDwf7n...,https://www.amazon.com/dp/B0B4689HLD,4.6,65,8.99,0.00,1,False,0
8,Beading & Jewelry Making,B09YRKCH71,Cinvo 40 Pcs Acrylic Pins Bulk Decoration Cute...,https://m.media-amazon.com/images/I/712QiTKefS...,https://www.amazon.com/dp/B09YRKCH71,4.4,61,9.99,0.00,1,False,50
9,Beading & Jewelry Making,B08ZXBT6RY,HUSUM 10 Pieces 6 Inches Curved Beading Needle...,https://m.media-amazon.com/images/I/71bfUVKYJq...,https://www.amazon.com/dp/B08ZXBT6RY,4.2,318,8.99,9.99,1,False,0


In [3]:
pd.options.display.max_rows = 4000
print(data['category_id'].value_counts())

category_id
91     28619
84     24660
270    20846
114    19822
118    18994
95     18514
110    18258
112    17679
116    17393
123    17005
105    16160
122    15626
173    15437
120    13921
113    12839
97     10938
88     10868
107     9580
51      9535
228     9485
72      9480
18      9418
16      9396
141     9364
10      9346
206     9254
71      9242
22      9238
143     9085
230     9020
179     8950
73      8859
43      8858
69      8823
168     8810
139     8777
147     8723
47      8669
12      8664
19      8659
142     8646
13      8611
163     8600
24      8536
174     8527
167     8523
209     8497
28      8487
64      8466
23      8444
63      8443
48      8402
3       8386
57      8364
8       8355
164     8337
180     8325
219     8313
21      8289
175     8283
60      8250
1       8246
6       8198
17      8195
165     8189
153     8146
68      8089
49      8074
138     8066
217     7973
154     7928
210     7854
140     7839
203     7794
223     7789
11      7778


In [4]:
category_id_to_keep_str = ['45','46','47','48','49','50','52','71','72','84','90','91','97','101','103','104','105','107','108','109','110','111','112','113','114','116','118','120','121','122','123','173','174', '270']
category_id_to_keep_str = ['122']
category_id_to_keep = [int(id) for id in category_id_to_keep_str]
filtered_data = data[data['category_id'].isin(category_id_to_keep)]
filtered_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15626 entries, 729269 to 744894
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   category_name      15626 non-null  object 
 1   asin               15626 non-null  object 
 2   title              15626 non-null  object 
 3   imgUrl             15626 non-null  object 
 4   productURL         15626 non-null  object 
 5   stars              15626 non-null  float64
 6   reviews            15626 non-null  int64  
 7   price              15626 non-null  float64
 8   listPrice          15626 non-null  float64
 9   category_id        15626 non-null  int64  
 10  isBestSeller       15626 non-null  bool   
 11  boughtInLastMonth  15626 non-null  int64  
dtypes: bool(1), float64(3), int64(3), object(5)
memory usage: 1.4+ MB


In [5]:
filtered_data.head(20)

,category_name,asin,title,imgUrl,productURL,stars,reviews,price,listPrice,category_id,isBestSeller,boughtInLastMonth
729269,Women's Shoes,B0B7WRQ373,Womens Winter Boots Warm: Womens Waterproof Sn...,https://m.media-amazon.com/images/I/61-dU-saLH...,https://www.amazon.com/dp/B0B7WRQ373,4.6,0,28.77,35.99,122,False,0
729270,Women's Shoes,B01LT99S30,Women's Lowpointed Low Heel Dress Pump Shoes,https://m.media-amazon.com/images/I/61men05KRx...,https://www.amazon.com/dp/B01LT99S30,4.4,0,47.99,69.99,122,False,0
729271,Women's Shoes,B0BKZHSDGP,Women's Elastic Chelsea Boots Chunky Block Hee...,https://m.media-amazon.com/images/I/71FVc143-u...,https://www.amazon.com/dp/B0BKZHSDGP,4.5,0,44.99,69.99,122,False,50
729272,Women's Shoes,B0B9FS85WY,Women's Snow Boots Anti-Slip Waterproof Outdoo...,https://m.media-amazon.com/images/I/710YF1P7kt...,https://www.amazon.com/dp/B0B9FS85WY,4.6,0,49.99,75.99,122,False,50
729273,Women's Shoes,B000J19AXS,Women's Tasman Slipper,https://m.media-amazon.com/images/I/811ATBGES0...,https://www.amazon.com/dp/B000J19AXS,4.7,0,109.95,0.00,122,False,2000
729274,Women's Shoes,B082HHRCBD,Women's Classic Ultra Mini Fashion Boot,https://m.media-amazon.com/images/I/816QYu6+Da...,https://www.amazon.com/dp/B082HHRCBD,4.6,0,149.95,0.00,122,False,1000
729275,Women's Shoes,B08WQ2BXP2,Women's Tazz Slipper,https://m.media-amazon.com/images/I/81nJHNKs5i...,https://www.amazon.com/dp/B08WQ2BXP2,4.6,0,129.95,0.00,122,False,1000
729276,Women's Shoes,B08WQ4PPL5,Women's Disquette Slipper,https://m.media-amazon.com/images/I/81GKFLAbaA...,https://www.amazon.com/dp/B08WQ4PPL5,4.5,0,109.95,0.00,122,False,1000
729277,Women's Shoes,B0BGJ46N2N,Women's Classic Ultra Mini Platform Fashion Boot,https://m.media-amazon.com/images/I/61MYjpwlDh...,https://www.amazon.com/dp/B0BGJ46N2N,4.6,0,159.95,0.00,122,False,900
729278,Women's Shoes,B09JVQQ99K,Women's Fuzzy Memory Foam Slippers Fluffy Wint...,https://m.media-amazon.com/images/I/71fnIod1Sq...,https://www.amazon.com/dp/B09JVQQ99K,4.6,0,19.99,32.99,122,False,7000


In [6]:
print(filtered_data['boughtInLastMonth'].value_counts())

boughtInLastMonth
0       12635
50       1608
100       830
200       247
300       105
400        54
1000       38
500        37
600        23
800        14
900        14
700        14
2000        5
3000        1
7000        1
Name: count, dtype: int64


In [7]:
filtered_data = filtered_data[~filtered_data['boughtInLastMonth'].isin([0])]


In [8]:
filtered_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2991 entries, 729271 to 744318
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   category_name      2991 non-null   object 
 1   asin               2991 non-null   object 
 2   title              2991 non-null   object 
 3   imgUrl             2991 non-null   object 
 4   productURL         2991 non-null   object 
 5   stars              2991 non-null   float64
 6   reviews            2991 non-null   int64  
 7   price              2991 non-null   float64
 8   listPrice          2991 non-null   float64
 9   category_id        2991 non-null   int64  
 10  isBestSeller       2991 non-null   bool   
 11  boughtInLastMonth  2991 non-null   int64  
dtypes: bool(1), float64(3), int64(3), object(5)
memory usage: 283.3+ KB


In [ ]:
filtered_data.head(20)

In [9]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv
import json

load_dotenv()

client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key = os.getenv("AZURE_OPENAI_API_KEY"),
    api_version = os.getenv("AZURE_OPENAI_VERSION")
)
deployment_name = os.getenv("AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME")
model_name = os.getenv("AZURE_OPENAI_COMPLETION_MODEL")

def create_summary(name, content):
    prompt = f"Create a simple but correct summary of an online shop product description based on the following details in less than 500 characters. Remove rating and pricing information and keep color, size and relevant configuration. If details are available describe how the product can be used and who the ideal customer for this product is. Do not add or use information that is not part of the prompt. Your description should be in english. This is the name of the product: {name} This are the details: {content}"

    response = client.chat.completions.create(
        model = model_name, 
        temperature=1.0,
        messages = [{"role" : "assistant", "content" : prompt}],
    )
    return response.choices[0].message.content


In [10]:
import requests
import json

def web_search_basic(
    name, subscription_key, auth_header_name="Ocp-Apim-Subscription-Key", mkt="en-us"    
):
    """Bing Web Search Basic REST call

    This sample makes a call to the Bing Web Search API with a text query and returns relevant pages
    Documentation: https://docs.microsoft.com/en-us/bing/search-apis/bing-web-search/overview

    May throw HTTPError in case of invalid parameters or a server error.

    Args:
        subscription_key (str): Azure subscription key of Bing Web Search service
        auth_header_name (str): Name of the authorization header
        query (str): Query to search for
        mkt (str): Market to search in
    """
    # Construct a request
    endpoint = "https://api.bing.microsoft.com/v7.0/search"
    params = {"q": "site:amazon.de what is the description of " + name, "mkt": mkt}
    headers = {auth_header_name: subscription_key}

    # Call the API
    try:
        response = requests.get(endpoint, headers=headers, params=params, timeout=10)
        response.raise_for_status()

        json_response = response.json()

        description = json_response["webPages"]["value"][0]["snippet"]

        for item in json_response["webPages"]["value"]:           
            description = item["snippet"] + " "

        summary = create_summary(name, description)
        summary = summary.replace("\n", "")
        return summary
    except Exception as ex:
        print(ex)
        print("++The above exception was thrown and handled succesfully++")
        return response


In [11]:
import time, sys
from IPython.display import clear_output

def update_progress(progress):
    bar_length = 20
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1

    block = int(round(bar_length * progress))
    clear_output(wait = True)
    text = "Progress: [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100)
    print(text)


subscription_key = os.getenv("BING_SUBSCRIPTION_KEY")

print(len(filtered_data))

limit = 550
counter = 0
# loop through the filtered data and search for the description of each product
for ind in filtered_data.head(limit).index:
    name = filtered_data['title'][ind]

    # print(name)

    description = web_search_basic(name, subscription_key)

    filtered_data.at[ind, 'description'] = description
    counter = counter + 1
    update_progress(counter / limit)


print("Processing complete!")

filtered_data.to_csv('../data/filtered_dataset_122_women_shoes.csv', index=False)

Progress: [####################] 100.0%
Processing complete!


In [ ]:
filtered_data.to_csv('../data/filtered_dataset.csv', index=False)